### Setup

In [1]:
import os
import json
import pandas as pd
import numpy as np
import sqlite3
from tqdm import tqdm
from collections import defaultdict
from contextlib import closing

In [2]:
path = '../../data/Datasus'

uf_dict = defaultdict(lambda: 'Invalid')

uf_dict[11] = 'RO'	
uf_dict[12] = 'AC'	
uf_dict[13] = 'AM'	
uf_dict[14] = 'RR'	
uf_dict[15] = 'PA'	
uf_dict[16] = 'AP'	
uf_dict[17] = 'TO'	
uf_dict[21] = 'MA'	
uf_dict[22] = 'PI'	
uf_dict[23] = 'CE'	
uf_dict[24] = 'RN'	
uf_dict[25] = 'PB'	
uf_dict[26] = 'PE'	
uf_dict[27] = 'AL'	
uf_dict[28] = 'SE'	
uf_dict[29] = 'BA'	
uf_dict[31] = 'MG'	
uf_dict[32] = 'ES'	
uf_dict[33] = 'RJ'	
uf_dict[35] = 'SP'	
uf_dict[41] = 'PR'	
uf_dict[42] = 'SC'	
uf_dict[43] = 'RS'	
uf_dict[50] = 'MS'	
uf_dict[51] = 'MT'	
uf_dict[52] = 'GO'	
uf_dict[53] = 'DF'
uf_dict[np.nan] = np.nan

In [3]:
files_list = list(os.listdir(path))

files_dict = defaultdict(list)

for file in files_list:
    if 'ANIMBR' in file: files_dict['ANIMBR'].append(file)
    elif 'CHAGBR' in file: files_dict['CHAGBR'].append(file)
    elif 'CHIKBR' in file: files_dict['CHIKBR'].append(file)
    elif 'DENGBR' in file: files_dict['DENGBR'].append(file)
    elif 'ESQUBR' in file: files_dict['ESQUBR'].append(file)
    elif 'HANSBR' in file: files_dict['HANSBR'].append(file)
    elif 'LEIVBR' in file: files_dict['LEIVBR'].append(file)
    elif 'LTANBR' in file: files_dict['LTANBR'].append(file)
    elif 'RAIVBR' in file: files_dict['RAIVBR'].append(file)


### Structure Analysis

In this step the structure of each csv was analysed, considering that the data pertaining to the same condition is planned to be stacked into a single database table first it was necessary to check if the data between each data source is consistent.

Unfortunately, exploration made clear that this consistent is not a guarantee. Therefore, a second step was made in which all possible columns for each condition were extracted in a dictionary and stored as a json to help with the <ins>Insertion</ins> step.

In [ ]:
def check_column_integrity(data_list:list, path:str=path, low_memory:bool=False) -> bool:

    base_columns = list(pd.read_csv(f'{path}/{data_list[0]}', encoding='ISO-8859-1', low_memory=low_memory, skiprows=lambda x: x not in [0]).columns)

    for file in data_list:
        column_list = list(pd.read_csv(f'{path}/{file}', encoding='ISO-8859-1', low_memory=low_memory, skiprows=lambda x: x not in [0]).columns)

        if not set(base_columns) == set(column_list):
            print(f' -- File Base = {data_list[0]}')
            print(f' -- File Falha = {file}')
            print(f'Diferenças 1: {set(base_columns) - set(column_list)}')
            print(f'Diferenças 2: {set(column_list) - set(base_columns)}')

            return False
        
    return True

In [ ]:
""" for key in files_dict:
    #if not key == 'DENGBR':
        print(f'{key}')
        print(f' -- {check_column_integrity(data_list=files_dict[key])}')
        print() """

In [14]:
full_columns = {}
for key in files_dict:
    all_columns = set()

    for file in tqdm(files_dict[key], total=len(files_dict[key])):
        all_columns |= set(list(pd.read_csv(f'{path}/{file}', encoding='ISO-8859-1', low_memory=False, skiprows=lambda x: x not in [0]).columns))

    full_columns[key] = all_columns

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:08<00:00,  1.23it/s]


In [ ]:
for key in full_columns: full_columns[key] = list(full_columns[key])
with open("../../data/Datasus/0_full_columns.json", 'w') as f: json.dump(full_columns, f)

## Data Insertion

In [ ]:
def process_instance(file:str, full_columns:set, name:str, con:sqlite3.Connection, aux_dict:dict=uf_dict, path:str=path, low_memory:bool=False) -> None:
    chunksize = 10 ** 6

    with pd.read_csv(f'{path}/{file}', encoding='ISO-8859-1', low_memory=low_memory, chunksize=chunksize) as reader:

            for chunk in reader:
                columns = set(list(chunk.columns))
                
                
                to_update = [column for column in columns if 'UF' in column and pd.api.types.is_any_real_numeric_dtype(chunk[column])]
                for column in to_update:
                    chunk[column] = chunk[column].map(aux_dict)

                
                columns_to_create = full_columns - columns

                for column in columns_to_create: 
                    chunk[column] = np.nan


                chunk.to_sql(name=name, con=con, if_exists='append')


def process_data(data_list:list, full_columns:set, name:str, con:sqlite3.Connection, aux_dict:dict=uf_dict, path:str=path, low_memory:bool=False) -> None:
    
    for file in tqdm(data_list, total=len(data_list)):
        process_instance(file=file, full_columns=full_columns, name=name, con=con)

### Insertions

I made the choice to insert the data of each table individually considering the possibility of a long execution time.

In [5]:
with open("../../data/Datasus/0_full_columns.json", 'r') as f:
        full_columns = json.load(f)

con = sqlite3.connect("../../data/base.db")

In [ ]:
# process_data(data_list=files_dict['ANIMBR'], full_columns=set(full_columns['ANIMBR']), name='peconhento_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['CHIKBR'], full_columns=set(full_columns['CHIKBR']), name='chikungunya_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['RAIVBR'], full_columns=set(full_columns['RAIVBR']), name='raiva_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['HANSBR'], full_columns=set(full_columns['HANSBR']), name='hanseniase_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['CHAGBR'], full_columns=set(full_columns['CHAGBR']), name='chagas_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['ESQUBR'], full_columns=set(full_columns['ESQUBR']), name='esquistossomose_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['DENGBR'], full_columns=set(full_columns['DENGBR']), name='dengue_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['LEIVBR'], full_columns=set(full_columns['LEIVBR']), name='leish_viceral_datasus', con=con)

In [ ]:
# process_data(data_list=files_dict['LTANBR'], full_columns=set(full_columns['LTANBR']), name='leish_tegumentar_datasus', con=con)

In [ ]:
## Close the connection
con.close()